# Import

In [1]:
import csv
import os
from glob import glob

import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import linalg as LA
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_mutual_info_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.mixture import GaussianMixture
from tqdm import tqdm

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


# Functions

In [2]:
def centralize_array(array):
    return array - np.mean(array, axis=0)

In [3]:
def normarize_array(array):
    return array / np.sqrt(np.sum(array * array, axis=1).reshape(-1, 1))

In [4]:
def normarize_vector(vector):
    return vector / np.sqrt(np.sum(vector * vector))

In [5]:
def get_average_vector(vectors):
    sum_vector = np.sum(vectors, axis=0)
    return normarize_vector(sum_vector)

# Read data

In [6]:
ag_news_df = pd.read_csv(
    "../data/CharCnn_Keras-master/data/ag_news_csv/train.csv",
    header=None,
    names=["class", "title", "description"],
)

In [7]:
label = ag_news_df["class"].to_numpy()

In [8]:
dim_nums = [2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 40, 80, 160, 320, 640]

In [9]:
class_nums = range(1, 5)

# 

In [10]:
model_num = 1

In [11]:
for count in tqdm(range(model_num)):
    MI = dict()
    AIC = dict()
    BIC = dict()
    LogL = dict()
    for dim_num in tqdm(dim_nums):
        # read vectors
        text_vectors = np.loadtxt(
            f"../DistanceMatrix/data/vector/{dim_num}.csv", delimiter=","
        )
        # preprocessing
        centralized_vectors = centralize_array(text_vectors)
        normarized_vectors = normarize_array(centralized_vectors)
        # get class distance matrix
        gmm = GaussianMixture(
            n_components=4, covariance_type="spherical", random_state=count
        )
        gmm.fit(normarized_vectors)
        pred = gmm.predict(normarized_vectors)
        # save prediction
        os.makedirs(f"./data/gmm_pred/{dim_num}/", exist_ok=True)
        with open(f"./data/gmm_pred/{dim_num}/{count}.csv", "w") as f:
            writer = csv.writer(f)
            writer.writerow(pred)
#         # get static values
#         MI[dim_num] = adjusted_mutual_info_score(label, pred)
#         AIC[dim_num] = gmm.aic(normarized_vectors)
#         BIC[dim_num] = gmm.bic(normarized_vectors)
#         LogL[dim_num] = gmm.score(normarized_vectors, label)
#     scores = pd.DataFrame(
#         [MI.values(), AIC.values(), BIC.values(), LogL.values()],
#         columns=dim_nums,
#         index=["MI", "AIC", "BIC", "LogLikelyhood"],
#     )
#     scores.to_csv(f"./data/gmm_score/{count}.csv")

100%|██████████| 1/1 [17:51<00:00, 1071.15s/it]
